# Updating Filevine Contact Geocodes

In [1]:
import numpy as np
import pandas as pd
import json
import httpx

In [2]:
# Load API credentials from secrets.json
with open("../secrets.json", "r") as file:
    secrets = json.load(file)

# Extract credentials
client_id = secrets["client_id"]
client_secret = secrets["client_secret"]
url = secrets["token_url"]
pat = secrets["pat_token"]

# Set up the data payload
data = {
    "grant_type": "personal_access_token",
    "token": pat,
    "client_id": client_id,
    "client_secret": client_secret,
    "scope": "fv.api.gateway.access tenant filevine.v2.api.* openid email fv.auth.tenant.read"
}
    
# Make the POST request
response = httpx.post(url, data=data, headers={"Content-Type": "application/x-www-form-urlencoded"})

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    token_info = response.json()
    access_token = token_info.get("access_token")
    # print("Access Token:", access_token)
    print("✅ Access Token obtained successfully")
else:
    print("❌ Failed to obtain access token:", response.text)  

# Define the API endpoint
USER_ORG_ENDPOINT = "https://api.filevineapp.com/fv-app/v2/utils/GetUserOrgsWithToken"

# Set up headers with the access token
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Make the POST request
response = httpx.post(USER_ORG_ENDPOINT, headers=headers)

# Check the response
if response.status_code == 200:
    user_info = response.json()
    user_id = user_info['user']['userId']['native']
    org_id = user_info['orgs'][0]['orgId']
    print(f"✅ User ID: {user_id}")
    print(f"✅ Org ID: {org_id}")
else:
    print(f"❌ Failed to retrieve User ID and Org ID: {response.text}")

# -----------------------------
# Configuration & API Endpoints
# -----------------------------

# Global headers including GZIP compression support.
headers = {
    "Authorization": f"Bearer {access_token}",
    "x-fv-userid": str(user_id),
    "x-fv-orgid": str(org_id),
    "Content-Type": "application/json",
    "Accept-Encoding": "gzip, deflate",
    "Accept": "application/json"
}

✅ Access Token obtained successfully
✅ User ID: 990018081
✅ Org ID: 990000379


In [3]:
df = pd.read_excel('./data/outbound_geolocated_reviewed.xlsx',
                    usecols = ['contactId', 'Full Name', 'Full Address','Latitude','Longitude'])
df

,contactId,Full Name,Full Address,Latitude,Longitude
0,996701857,Waldorf Total Health Chiropractic & Physical T...,"12102 Old Line Center, Waldorf, MD 20602",38.616638,-76.890698
1,996605635,Bezak Chiropractic And Rehabilitation,"7500 Hanover Parkway, Greenbelt, MD 20770",38.992915,-76.875826
2,996356411,Dunkirk Chiropractic & Wellness Center,"10020 Southern Maryland Blvd, Dunkirk, MD 20754",38.714689,-76.659379
3,996605523,"Effective Integrative Healthcare - Lanham, Md","7400 Riverdale Rd, Lanham, MD 20706",38.962083,-76.884654
4,996616309,Rebound Integrative Health & Sports Clinic,"7701 Greenbelt Rd, Greenbelt, MD 20770-6521",38.994033,-76.874777
...,...,...,...,...,...
58,996701927,"Maryland Healthcare Clinics - Owings Mills, Md","5 Park Center Court, Owings Mills, MD 21117",39.399382,-76.753732
59,996465882,Washington Wellness,"25 Massachusetts Ave Nw, Washington, DC 20001-...",38.898473,-77.011358
60,996593018,Neuropro Concussion Clinic,"1110 Benfield Blvd, Millersville, MD 21108",39.094464,-76.631838
61,996668361,Rebound Integrative Health & Sports Clinic - T...,"3611 Branch Avenue, Temple Hills, MD 20748",38.844634,-76.950739


In [4]:
## Use Sample Person ID for review
contactId = 996701857
endpoint = f"https://api.filevineapp.com/fv-app/v2/Contacts/{contactId}"

# Send the PATCH request
response = httpx.get(endpoint, headers=headers)

# Check the response
if response.status_code == 200:
    print("✅ Fields retrieved successfully:", response.json())
else:
    print(f"❌ Failed to retrieve fields: {response.status_code} - {response.text}")

✅ Fields retrieved successfully: {'personId': {'native': 996701857, 'partner': None}, 'firstName': 'Waldorf Total Health Chiropractic & Physical Therapy', 'middleName': '', 'lastName': '', 'isSingleName': True, 'fullName': 'Waldorf Total Health Chiropractic & Physical Therapy', 'pictureUrl': '/images/Defaulta63d0804-0352-4277-b8e2-4eaead2775e9.png', 'pictureKey': 'Defaulta63d0804-0352-4277-b8e2-4eaead2775e9.png', 'fromCompany': '', 'abbreviatedName': 'Total Health Chiropractic & Physical Therapy', 'personTypes': ['MedicalProvider'], 'uniqueId': 'ded34dbe-bba6-4614-949d-8ec53c34a1cc', 'searchNames': ['waldorf', 'total', 'health', 'chiropractic', '&', 'physical', 'therapy', 'waldorf total health chiropractic & physical therapy', 'total health chiropractic & physical therapy'], 'phones': [{'phoneId': {'native': 998552389, 'partner': None}, 'number': '(240) 754-7130', 'rawNumber': '2407547130', 'label': 'Work', 'links': {}}, {'phoneId': {'native': 998552390, 'partner': None}, 'number': '(2

In [5]:
endpoint = "https://api.filevineapp.com/fv-app/v2/Custom-Contacts-Meta"

# Send the PATCH request
response = httpx.get(endpoint, headers=headers, params = {'onlyCustomFields':True})

# Check the response
if response.status_code == 200:
    print("✅ Fields retrieved successfully.")#, response.json())
else:
    print(f"❌ Failed to retrieve fields: {response.status_code} - {response.text}")

✅ Fields retrieved successfully.


In [6]:
len(response.json())

28

In [7]:
response.json()

[{'fieldName': 'First Name Pronunciation',
  'selector': 'custom.firstNamePronunciation',
  'action': 'UPDATE|REMOVE',
  'value': 'string',
  'fieldType': 'String',
  'tab': {'id': 990000494, 'title': 'Details'},
  'fieldset': {'id': 990002398, 'title': None},
  'limitByPersonType': [990007689,
   990007690,
   990007691,
   990007692,
   990007693,
   990007695,
   990007696,
   990007697,
   990007698,
   990008375,
   990008587],
  'examples': [{'selector': 'custom.firstNamePronunciation',
    'action': 'UPDATE',
    'value': 'string'},
   {'selector': 'custom.firstNamePronunciation', 'action': 'REMOVE'}]},
 {'fieldName': 'Last Name Pronunciation',
  'selector': 'custom.lastNamePronunciation',
  'action': 'UPDATE|REMOVE',
  'value': 'string',
  'fieldType': 'String',
  'tab': {'id': 990000494, 'title': 'Details'},
  'fieldset': {'id': 990002398, 'title': None},
  'limitByPersonType': [990007689,
   990007690,
   990007691,
   990007692,
   990007693,
   990007695,
   990007696,
   9

In [8]:
for item in response.json():
    if item.get('fieldName', {}) in ['Latitude', 'Longitude']:
        print(
            'Field Name:',item.get('fieldName'), '|',
            'Selector:', item.get('selector', None), '|',
            'Value:', item.get('value', None), '|',
            'Field Type:',item.get('fieldType'), '|',
            'Action:', item.get('action', None))
    else:
        pass

Field Name: Latitude | Selector: custom.latitude2 | Value: string | Field Type: String | Action: UPDATE|REMOVE
Field Name: Longitude | Selector: custom.longitude2 | Value: string | Field Type: String | Action: UPDATE|REMOVE


In [14]:
## Specify endpoint
contactId = 996701857
endpoint = f"https://api.filevineapp.com/fv-app/v2/Custom-Contacts/{contactId}"


action = 'Update'

# id = 'latitude'
selector = 'custom.latitude2'
value = "38.616638"

# id = 'Longitude'
# selector = 'custom.longitude'
# value = -76.890698

## Create payload to update the field
payload = {
    'Selector': selector,
    'Action': action,
    'Value':value}

# Send the PATCH request
response = httpx.patch(endpoint, headers=headers, json=payload)

# Check the response
if response.status_code == 200:
    print("✅ Field updated successfully:", response.json())
else:
    print(f"❌ Failed to update field: {response.status_code} - {response.text}")

❌ Failed to update field: 500 - 


In [10]:
response

<Response [500 Internal Server Error]>